# EIMLIA-3M-TEU - Guide de démarrage rapide

Ce notebook présente les principales fonctionnalités du projet EIMLIA-3M-TEU:
- Entraînement des modèles d'IA (TRIAGEMASTER, URGENTIAPARSE, EMERGINET)
- Explicabilité avec SHAP
- Simulation du service d'urgences
- Process Mining avec PM4Py

In [ ]:
# Installation (si nécessaire)
# !pip install -e ..

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Chargement des données

In [ ]:
from src.models.data_loader import generate_synthetic_data, load_data

# Générer des données synthétiques pour la démo
texts, numerical, labels, feature_names = generate_synthetic_data(500)

print(f"Nombre d'échantillons: {len(texts)}")
print(f"Features: {feature_names}")
print(f"Distribution des labels: {np.bincount(labels)}")

In [ ]:
# Exemple de verbatim
for i in range(3):
    print(f"\n[Label {labels[i]+1}] {texts[i][:100]}...")

## 2. Entraînement d'un modèle

In [ ]:
from sklearn.model_selection import train_test_split

# Split des données
train_t, val_t, train_n, val_n, train_l, val_l = train_test_split(
    texts, numerical, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"Train: {len(train_t)} | Validation: {len(val_t)}")

In [ ]:
from src.models import TRIAGEMASTER

# Entraîner TRIAGEMASTER (le plus rapide)
model = TRIAGEMASTER(
    doc2vec_dim=50,  # Réduit pour la démo
    epochs=20,
    patience=5
)

model.fit(
    train_t, train_n, train_l, feature_names,
    val_data=(val_t, val_n, val_l)
)

In [ ]:
# Prédictions
predictions = model.predict(val_t, val_n)

# Évaluation
from src.models.evaluation import evaluate_model

results = evaluate_model(val_l, predictions, model_name='TRIAGEMASTER')

## 3. Explicabilité SHAP

In [ ]:
# Expliquer une prédiction
idx = 0
shap_values, feature_importance = model.explain_shap(
    val_t[idx:idx+1],
    val_n[idx:idx+1]
)

print("\n📊 Importance des features:")
for name, imp in sorted(feature_importance.items(), key=lambda x: -x[1])[:5]:
    print(f"  {name}: {imp:.3f}")

## 4. Simulation du service d'urgences

In [ ]:
from src.simulation.config import SimulationConfig
from src.simulation.patient_generator import GenerateurPatientsVirtuels

# Configuration
config = SimulationConfig(duree_jours=1)

# Générer une cohorte de patients
gen = GenerateurPatientsVirtuels(config, seed=42)
patients = gen.generer_cohorte(100, verbose=True)

In [ ]:
# Visualiser la distribution
df_patients = gen.to_dataframe(patients)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution FRENCH
df_patients['niveau_french_reel'].hist(ax=axes[0], bins=5, edgecolor='black')
axes[0].set_xlabel('Niveau FRENCH')
axes[0].set_title('Distribution des niveaux')

# Distribution âge
df_patients['age'].hist(ax=axes[1], bins=20, edgecolor='black')
axes[1].set_xlabel('Âge')
axes[1].set_title('Distribution des âges')

# SpO2
df_patients['spo2'].hist(ax=axes[2], bins=20, edgecolor='black')
axes[2].set_xlabel('SpO2 (%)')
axes[2].set_title('Distribution SpO2')

plt.tight_layout()
plt.show()

In [ ]:
from src.simulation.simpy_des import SimulationUrgencesSimPy

# Simulation courte (1 jour)
config = SimulationConfig(duree_jours=1, agent_ia='emerginet')
sim = SimulationUrgencesSimPy(config, verbose=True)

results = sim.run()

print(f"\n📊 Résultats:")
print(f"  Patients traités: {results['n_patients']}")
print(f"  DMS médiane: {results['dms_median']:.1f} min")
print(f"  Attente triage: {results['attente_triage_median']:.1f} min")

## 5. Comparaison des scénarios

In [ ]:
from src.simulation.hybrid import run_scenario

# Comparer différents scénarios (sur 1 jour pour la démo)
scenarios = ['reference', 'nlp', 'jepa']
resultats = {}

config_base = SimulationConfig(duree_jours=1)

for scenario in scenarios:
    print(f"\n{'='*50}")
    print(f"Scénario: {scenario}")
    resultats[scenario] = run_scenario(scenario, config_base, verbose=False)
    print(f"DMS médiane: {resultats[scenario].get('dms_mean', 0):.1f} min")

In [ ]:
# Visualisation comparative
from src.utils.visualization import create_summary_table

df_comparison = create_summary_table(resultats)
df_comparison

## 6. Process Mining

In [ ]:
# Note: Nécessite PM4Py installé et un event log
# from src.process_mining import ProcessMiningPipeline
# 
# pipeline = ProcessMiningPipeline('data/event_logs/traces.csv')
# kpis = pipeline.compute_kpis()
# print(kpis)

## Conclusion

Ce notebook a présenté les principales fonctionnalités d'EIMLIA-3M-TEU:

1. **Modèles d'IA**: 3 architectures avec explicabilité intégrée
2. **Simulation**: DES (SimPy) + SMA (Mesa) hybride
3. **Process Mining**: Analyse des parcours avec PM4Py

Pour une analyse complète, utilisez:
```bash
python scripts/run_simulation.py --scenario all --days 180
```